# Homework 18: Maximum Likelihood

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import utils as utils
from fractions import Fraction

from IPython.display import display, HTML

# Inline plotting
%matplotlib inline

# Make sympy print pretty math expressions
sy.init_printing()

utils.load_custom_styles()

---
## Exercise 3.1

<img src="figures/homework-18/exercise-3.1.png" width="600" />




















In [2]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])

Compute the prior probabilities for the classes. We can compute the prior probability for each class $c_k$ as follows:

$$
P(c_k) = \frac{N_k}{\sum_l^K N_l}
$$
where $N_k$ is the number of samples in class $c_k$

Class $c_1$ has 7 samples and $c_2$ has 5 samples. In total, we have 12 samples. Therefore, $P(c_1) = 7/12=0.58$ and $P(c_2) = 5/12=0.42$.

In [3]:
N1 = c1.shape[1]
N2 = c2.shape[1]
N_sum = N1 + N2

P_c1 = N1/N_sum
P_c2 = N2/N_sum
P_c1, P_c2

We compute the class-conditional probabilities in multiple steps:
$$
p(\mathbf{x} \mid c_k) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)  }
  {  \sum_{l=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_l \rVert_2)   }
$$

Step 1: Compute the mean vector associated with each class:

In [4]:
m1 = np.mean(c1, axis=1).reshape(-1, 1)
m1

array([[-0.85714286],
       [-0.9       ]])

In [5]:
m2 = np.mean(c2, axis=1).reshape(-1, 1)
m2

array([[1.1],
       [1. ]])

Step 2: Compute $\exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)$

In [6]:
c1_numerator = np.exp(-np.linalg.norm(X - m1, axis=0))
c1_numerator

array([0.28855859, 0.07016722, 0.40201441, 0.18136432, 0.0830477 ])

Step 3: compute $\exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)$:

In [7]:
c2_numerator = np.exp(-np.linalg.norm(X - m2, axis=0))
c2_numerator

array([0.22613867, 0.90483742, 0.09769151, 0.2822644 , 0.24836923])

Step 4: Compute the class-condition probability for class 1:

$$
p(\mathbf{x} \mid c_1) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)  }
  {  \sum_{k=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)   }
$$

In [8]:
p_x_c1 = c1_numerator / (c1_numerator + c2_numerator)
p_x_c1

array([0.5606375 , 0.07196604, 0.804502  , 0.39118439, 0.25058376])

Step 5: Compute the class-condition probability for class 2:

$$
p(\mathbf{x} \mid c_2) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_2 \rVert_2)  }
  {  \sum_{k=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)   }
$$

In [9]:
p_x_c2 = c2_numerator / (c1_numerator + c2_numerator)
p_x_c2

array([0.4393625 , 0.92803396, 0.195498  , 0.60881561, 0.74941624])

Step 7: We can now compute conditional probability, which has the following formula (Bayes' formula):

$$
P(c_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid c_k) P(c_k)}{p(\mathbf{x})}
$$
where
$$
p(\mathbf{x}) = \sum_{k=1}^K p(\mathbf{x} \mid c_k) P(c_k)
$$

Since $p(\mathbf{x})$ is the same for when computing $P(c_k\mid \mathbf{x})$, it is not necessary to compute it when we want to use the conditional probability to classify new samples. Therefore, we omit the division:
$$
P(c_k \mid \mathbf{x}) = p(\mathbf{x} \mid c_k) P(c_k)
$$

In [11]:
P_c1_x = p_x_c1 * P_c1
P_c1_x

array([0.32703854, 0.04198019, 0.46929284, 0.2281909 , 0.14617386])

In [12]:
P_c2_x = p_x_c2 * P_c2
P_c2_x

array([0.18306771, 0.38668082, 0.0814575 , 0.25367317, 0.31225677])

Step 8: Once we have the condition probabilities, we can use Bayes' Decision Rule to classify our samples $X$:

In [13]:
# Create a matrix so we can easily compute argmin
P_ck_x = np.concatenate([[P_c1_x], 
                         [P_c2_x]])
P_ck_x

array([[0.32703854, 0.04198019, 0.46929284, 0.2281909 , 0.14617386],
       [0.18306771, 0.38668082, 0.0814575 , 0.25367317, 0.31225677]])

In [14]:
np.argmax(P_ck_x, axis=0)+1

array([1, 2, 1, 2, 2])

So our classification is as follows: 

\begin{align}
c_1 &= \{ \mathbf{x}_1, \mathbf{x}_3 \} \\
c_2 &= \{ \mathbf{x}_2, \mathbf{x}_4, \mathbf{x}_5 \}
\end{align}

Below is a function that can compute the conditional probability for any number of classes.

In [51]:
def compute_condition_prob(C, X, verbose=True):
    n_classes = len(C)
    N_ck = []
    p_x_ck_numerator = []
    for c in C:
        N_ck.append(c.shape[1])
        mu_k = np.mean(c, axis=1).reshape(-1, 1)
        p_x_ck_numerator.append(np.exp(-np.linalg.norm(X - mu_k, axis=0)))
    P_ck = []
    p_x_ck = []
    P_ck_x = []
    for i in range(n_classes):
        P_ck.append(N_ck[i] / np.sum(N_ck))
        p_x_ck.append(p_x_ck_numerator[i] / np.sum(p_x_ck_numerator, axis=0))
        P_ck_x.append(p_x_ck[i] * P_ck[i])
    if verbose:
        for i in range(n_classes):
            print('P(c{}): {}'.format(i+1, str(P_ck[i])))
        print('---')
        for i in range(n_classes):
            print('p(x|c{}): {}'.format(i+1, str(p_x_ck[i])))
        print('---')
        for i in range(n_classes):
            print('P(c{}|x): {}'.format(i+1, str(P_ck_x[i])))
    return {
        'P_ck': P_ck,
        'p_x_ck': p_x_ck,
        'P_ck_x': P_ck_x
    }

In [52]:
result = compute_condition_prob([c1, c2], X)

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [0.5606375  0.07196604 0.804502   0.39118439 0.25058376]
p(x|c2): [0.4393625  0.92803396 0.195498   0.60881561 0.74941624]
---
P(c1|x): [0.32703854 0.04198019 0.46929284 0.2281909  0.14617386]
P(c2|x): [0.18306771 0.38668082 0.0814575  0.25367317 0.31225677]


  ---
## Exercise 3.2

<img src="figures/homework-18/exercise-3.2.png" width="600" />




















In [55]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])
Lambda = np.array([[0, 1],
                   [1, 0]])

In [63]:
Lambda[:,0].reshape(-1, 1)

array([[0],
       [1]])

In [66]:
result['P_ck_x']

[array([0.32703854, 0.04198019, 0.46929284, 0.2281909 , 0.14617386]),
 array([0.18306771, 0.38668082, 0.0814575 , 0.25367317, 0.31225677])]

In [64]:
def risk_based_classifier(C, Lambda, X):
    result = compute_condition_prob(C, X)
    P_ck_x = result['P_ck_x']
    
    n_classes = len(P_ck_x)
    n_actions = Lambda.shape[1]
    R_ai_x = []
    for i in range(n_actions):
        for k in range(n_classes):
            lmbda = Lambda[:,k].reshape(-1, 1)

    R_ck_x = np.concatenate([[R_c1_x], 
                             [R_c2_x]])
    return np.argmin(R_ck_x, axis=0)+1

In [65]:
risk_based_classifier([c1, c2], Lambda, X)

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [0.5606375  0.07196604 0.804502   0.39118439 0.25058376]
p(x|c2): [0.4393625  0.92803396 0.195498   0.60881561 0.74941624]
---
P(c1|x): [0.32703854 0.04198019 0.46929284 0.2281909  0.14617386]
P(c2|x): [0.18306771 0.38668082 0.0814575  0.25367317 0.31225677]


ValueError: shapes (2,1) and (5,) not aligned: 1 (dim 1) != 5 (dim 0)

So our classification is as follows: 

\begin{align}
c_1 &= \{ \mathbf{x}_1, \mathbf{x}_3 \} \\
c_2 &= \{ \mathbf{x}_2, \mathbf{x}_4, \mathbf{x}_5 \} \\
\end{align}

---
## Exercise 3.3

<img src="figures/homework-18/exercise-3.3.png" width="600" />







In [15]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])
Lambda = np.array([[0.3, 0.8],
                   [0.7, 0.2]])

In [18]:
risk_based_classifier(c1, c2, Lambda, X)

array([1, 2, 1, 2, 2])

So our classification is as follows: 

\begin{align}
c_1 &= \{ \mathbf{x}_1, \mathbf{x}_3 \} \\
c_2 &= \{ \mathbf{x}_2, \mathbf{x}_4, \mathbf{x}_5 \} \\
\end{align}

---
## Exercise 3.4

<img src="figures/homework-18/exercise-3.4a.png" width="600" />





<img src="figures/homework-18/exercise-3.4b.png" width="600" />














In [20]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])
sigma_c1 = np.array([[1, 0],
                     [0, 2]])
sigma_c2 = np.array([[1, 0],
                     [0, 2]])

---
## Exercise 3.5

<img src="figures/homework-18/exercise-3.5.png" width="600" />











